In [7]:
# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

# Torchtest package
# import torchtext
# from torchtext.datasets import Multi30k
# from torch.utils.data import DataLoader
# from torchtext.data.utils import get_tokenizer
# from collections import Counter
# from torchtext.vocab import vocab
# from torchtext.utils import download_from_url, extract_archive
# from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

torch.seed

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<function torch.random.seed() -> int>

In [8]:
# Check device availability
device = ''
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
device = torch.device(device)
print("You are using device: %s" % device)

You are using device: mps


In [9]:
encoder_output = torch.rand((5, 320, 20)).to(device) # N, input_size, T

In [10]:
from Decoder import Decoder
from PostNet import PostNet

In [11]:
decoder = Decoder(lstm_bidirectional=True).to(device)
decoder_output = decoder.forward(encoder_output)
print(decoder_output.shape) # N, output_size, T
print(decoder_output)

torch.Size([5, 256, 20])
tensor([[[ 0.0049,  0.0212, -0.0038,  ..., -0.0357, -0.0692, -0.0574],
         [-0.0401, -0.0437, -0.0136,  ...,  0.0015, -0.0009,  0.0054],
         [ 0.0180, -0.0018, -0.0139,  ...,  0.0046,  0.0354, -0.0194],
         ...,
         [ 0.0125,  0.0148,  0.0717,  ...,  0.0401,  0.0533,  0.0581],
         [ 0.0380,  0.0231,  0.0695,  ..., -0.0339, -0.0103,  0.0286],
         [ 0.0029,  0.0173,  0.0301,  ...,  0.0196,  0.0051, -0.0239]],

        [[-0.0362, -0.0336, -0.0326,  ..., -0.0140, -0.0367, -0.0419],
         [ 0.0271,  0.0553,  0.0397,  ...,  0.0463,  0.0378,  0.0515],
         [ 0.0624,  0.0914,  0.0804,  ...,  0.0506,  0.0346, -0.0216],
         ...,
         [ 0.0305,  0.0502,  0.0677,  ...,  0.0088,  0.0577,  0.0511],
         [ 0.0214, -0.0062,  0.0041,  ...,  0.0028,  0.0246,  0.0548],
         [ 0.0028,  0.0415,  0.0725,  ...,  0.0605,  0.0695,  0.0339]],

        [[-0.0173, -0.0301, -0.0227,  ..., -0.0217, -0.0529, -0.0655],
         [-0.0011,  

In [6]:
postnet = PostNet().to(device)
postnet_output = postnet.forward(decoder_output)
print(postnet_output.shape) # (N, output size, T)
print(postnet_output)

RuntimeError: linear(): input and weight.T shapes cannot be multiplied (256x20 and 256x256)

In [9]:
# Loss Test
from utils import criterion
#
input = torch.rand((5, 80, 1)).to(device)

print(input.shape)
print(encoder_output.shape)
print(decoder_output.shape)
print(postnet_output.shape)

criterion(input=input, encoder_ouput=encoder_output, decoder_output=decoder_output, postnet_output=postnet_output, encoded_postnet_output=encoder_output)

torch.Size([5, 80, 1])
torch.Size([5, 320, 20])
torch.Size([5, 80, 20])
torch.Size([5, 80, 20])


/opt/homebrew/anaconda3/envs/cs7643-finalproject-v2/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([5, 80, 20])) that is different to the input size (torch.Size([5, 80, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.7716, device='mps:0', grad_fn=<AddBackward0>)

# Training, evaluation, and plotting loss curves

In [1]:
# Code provide to you for training and evaluation
from utils import train
from utils import evaluate
from utils import plot_curves
from Model import Model

def train_and_plot(model, optimizer, scheduler, criterion, train_loader, valid_loader, filename, epochs, device='cpu'):
    train_loss_history = []
    valid_loss_history = []

    for epoch_idx in range(epochs):
        print("-----------------------------------")
        print("Epoch %d" % (epoch_idx+1))
        print("-----------------------------------")

        train_loss, avg_train_loss = train(model, train_loader, optimizer, criterion, device=device)
        scheduler.step(train_loss)

        val_loss, avg_val_loss = evaluate(model, valid_loader, criterion, device=device)

        train_loss_history.append(avg_train_loss)
        valid_loss_history.append(avg_val_loss)

        print("Training Loss: %.4f. Validation Loss: %.4f." % (avg_train_loss, avg_val_loss))

    plot_curves(train_loss_history, valid_loss_history, filename)


In [ ]:

# Example usage:
model = Model()
train_loader = DataLoader(your_train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(your_valid_dataset, batch_size=32, shuffle=False)
optimizer = Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
criterion = criterion(input=input, encoder_ouput=encoder_output, decoder_output=decoder_output, postnet_output=postnet_output, encoded_postnet_output=encoder_output)
filename = "training_loss_curves"
epochs = 10
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_and_plot(model, optimizer, scheduler, criterion, train_loader, valid_loader, filename, epochs, device)